# Battle of the Neighborhood
This project will utilize the location data from Foursquare by making RESTful API calls to the Foursquare API to retrieve data about venues in different neighborhoods around the world. The data are not readily available by scraping web data and parsing HTML code. This project will utilize Python and its pandas library to manipulate data, exploring and analyzing data. 


In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Using FourSquare API with Python
Consturct a URL to send to the API to search for a specific type of venues, to explore a particular vanue, to explore a user, to explore a geographical location, or get trending venues around a location.
Then use Folium to visualize the results.

In [3]:
#!conda install -c conda-forge geopy --yes
#print("geopy installed")
!conda install -c conda-forge folium=0.5.0 --yes
print("folium installed")

import requests
import random
from geopy.geocoders import Nominatim

# for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# for transforming json file into a panda dataframe
from pandas.io.json import json_normalize

# for visualization
import folium

print('Libraries imported')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::anaconda==2019.07=py37_0
  - defaults/osx-64::numba==0.44.1=py37h6440ff4_0
done

## Package Plan ##

  environment location: /Library/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           6 KB
    altair-3.2.0               |           py37_0         773 KB  conda-forge
    anaconda-custom            |           py37_1           3 KB
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           

## set up credentials and version

In [4]:
CLIENT_ID = "S3MHJEDILLZFRYQQUJBMPMSO35SOVO3MP13KCNO3TJHIVFYC"
CLIENT_SECRET = "EGTGKCFNBVMKAUPAGH1E1ZUZCFYT54AT3JCMVZOLNQ3KZQOT"
VERSION = "20190818"
LIMIT = 30

## use geopy function to convert address to latitude/longitude
input: address [string]; output: (latitude, longitude) [float, float]

example input "102 North End Ave, New York, NY"

In [5]:
def convert_address(address):
    geolocator = Nominatim(user_agent = "foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return (latitude, longitude)

In [7]:
# example for Contrad Hotel
address = "102 North End Ave, New York, NY"
latitude, longitude = convert_address(address)
print((latitude, longitude))

(40.7149555, -74.0153365)


## search for a specific venue category

In [15]:
#CLIENT_ID = "S3MHJEDILLZFRYQQUJBMPMSO35SOVO3MP13KCNO3TJHIVFYC"
#CLIENT_SECRET = "EGTGKCFNBVMKAUPAGH1E1ZUZCFYT54AT3JCMVZOLNQ3KZQOT"
#VERSION = "20190818"
#LIMIT = 30

def search_venue_category(search_query, radius, latitude, longitude, CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, VERSION=VERSION, LIMIT=LIMIT):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results=requests.get(url).json() # returns a dict
    #print(results)
    venues = results["response"]["venues"]
    
    #transform into pandas dataframe
    df = json_normalize(venues)
    #print(df.head())
    
    # keep only categories, location info
    filtered_columns = ['name', 'categories', 'id'] + [col for col in df.columns if col.startswith('location.')]
    dataframe_filtered = df.loc[:, filtered_columns]
    
    #function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
        
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    
    return dataframe_filtered


In [20]:
# example for searching for Japenese Restaurants near Contrad Hotel
search_query, radius = "Japanese", 500
dataframe_filtered = search_venue_category(search_query, radius, latitude, longitude)
(dataframe_filtered)

,name,categories,id,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state
0,Yoshi Japanese Restaurant,Japanese Restaurant,453a7f23f964a520ee3b1fe3,201 E Houston St,US,New York,United States,Ludlow,210,"[201 E Houston St (Ludlow), New York, NY 10002...","[{'label': 'display', 'lat': 40.71313909682906...",40.713139,-74.016032,NaN,10002,NY
1,Korin,Furniture / Home Store,4af5d65ff964a52091fd21e3,57 Warren St,US,New York,United States,Church St,500,"[57 Warren St (Church St), New York, NY 10007,...","[{'label': 'display', 'lat': 40.71482437714839...",40.714824,-74.009404,Tribeca,10007,NY


## Visualize the category search result

In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel
# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map